# LeaderBoard

In [1]:
import autoroot
import typing as tp
from dataclasses import dataclass
import numpy as np
import pandas as pd
import xarray as xr
import einops
import metpy
from metpy.units import units
import xarray_dataclasses as xrdataclass
from oceanbench._src.datasets.base import XRDABatcher
from oceanbench._src.geoprocessing import geostrophic as geocalc
import matplotlib.pyplot as plt
import seaborn as sns

sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)

%load_ext autoreload
%autoreload 2


In [4]:
results = pd.read_csv("/gpfswork/rech/yrf/commun/data_challenges/dc20a_osse/staging/results/leaderboard.csv", index_col=0)

In [17]:
cond1 = results["Variable"] == "SEA_SURFACE_HEIGHT"

## NADIR4

In [18]:
cond2 = results["Experiment"] == "NADIR4"
results.loc[cond1 & cond2].sort_values("λx [km]")

,Experiment,Method,Variable,µ(RMSE),σ(RMSE),iso λx [km],iso λx [degree],λx [km],λx [degree],λt [days]
0,NADIR4,4DVARNET,SEA_SURFACE_HEIGHT,0.938075,0.006256,71.736684,0.646276,108.714515,0.979410,7.733046
0,NADIR4,4DVARNET,SEA_SURFACE_HEIGHT,0.938075,0.006256,71.736684,0.646276,108.714515,0.979410,7.733046
0,NADIR4,BFNQG,SEA_SURFACE_HEIGHT,0.924509,0.011475,95.774617,0.862834,141.222640,1.272276,10.987376
0,NADIR4,BFNQG,SEA_SURFACE_HEIGHT,0.924509,0.011475,95.774617,0.862834,141.222640,1.272276,10.987376
0,NADIR4,BFNQG,SEA_SURFACE_HEIGHT,0.924509,0.011475,95.774617,0.862834,141.222640,1.272276,10.987376
0,NADIR4,BFNQG,SEA_SURFACE_HEIGHT,0.924509,0.011475,95.774617,0.862834,141.222640,1.272276,10.987376
0,NADIR4,DYMOST,SEA_SURFACE_HEIGHT,0.912157,0.012019,116.965030,1.053739,156.229825,1.407476,11.486388
0,NADIR4,DYMOST,SEA_SURFACE_HEIGHT,0.912157,0.012019,116.965030,1.053739,156.229825,1.407476,11.486388
0,NADIR4,MIOST,SEA_SURFACE_HEIGHT,0.928909,0.005333,102.379241,0.922336,157.892636,1.422456,10.668045
0,NADIR4,MIOST,SEA_SURFACE_HEIGHT,0.928909,0.005333,102.379241,0.922336,157.892636,1.422456,10.668045


In [20]:
results.loc[cond1 & cond2].sort_values("λt [days]")

,Experiment,Method,Variable,µ(RMSE),σ(RMSE),iso λx [km],iso λx [degree],λx [km],λx [degree],λt [days]
0,NADIR4,4DVARNET,SEA_SURFACE_HEIGHT,0.938075,0.006256,71.736684,0.646276,108.714515,0.979410,7.733046
0,NADIR4,4DVARNET,SEA_SURFACE_HEIGHT,0.938075,0.006256,71.736684,0.646276,108.714515,0.979410,7.733046
0,NADIR4,MIOST,SEA_SURFACE_HEIGHT,0.928909,0.005333,102.379241,0.922336,157.892636,1.422456,10.668045
0,NADIR4,MIOST,SEA_SURFACE_HEIGHT,0.928909,0.005333,102.379241,0.922336,157.892636,1.422456,10.668045
0,NADIR4,BFNQG,SEA_SURFACE_HEIGHT,0.924509,0.011475,95.774617,0.862834,141.222640,1.272276,10.987376
0,NADIR4,BFNQG,SEA_SURFACE_HEIGHT,0.924509,0.011475,95.774617,0.862834,141.222640,1.272276,10.987376
0,NADIR4,BFNQG,SEA_SURFACE_HEIGHT,0.924509,0.011475,95.774617,0.862834,141.222640,1.272276,10.987376
0,NADIR4,BFNQG,SEA_SURFACE_HEIGHT,0.924509,0.011475,95.774617,0.862834,141.222640,1.272276,10.987376
0,NADIR4,DYMOST,SEA_SURFACE_HEIGHT,0.912157,0.012019,116.965030,1.053739,156.229825,1.407476,11.486388
0,NADIR4,DYMOST,SEA_SURFACE_HEIGHT,0.912157,0.012019,116.965030,1.053739,156.229825,1.407476,11.486388


## SWOT 1 + NADIR 5

In [21]:
cond3 = results["Experiment"] == "SWOT1NADIR5"
results.loc[cond1 & cond3].sort_values("λx [km]")

,Experiment,Method,Variable,µ(RMSE),σ(RMSE),iso λx [km],iso λx [degree],λx [km],λx [degree],λt [days]
0,SWOT1NADIR5,4DVARNET,SEA_SURFACE_HEIGHT,0.950371,0.009098,54.457253,0.490606,75.947262,0.684210,5.818690
0,SWOT1NADIR5,4DVARNET,SEA_SURFACE_HEIGHT,0.950371,0.009098,54.457253,0.490606,75.947262,0.684210,5.818690
0,SWOT1NADIR5,BFNQG,SEA_SURFACE_HEIGHT,0.935132,0.014810,69.443484,0.625617,118.899047,1.071163,36.701605
0,SWOT1NADIR5,BFNQG,SEA_SURFACE_HEIGHT,0.935132,0.014810,69.443484,0.625617,118.899047,1.071163,36.701605
0,SWOT1NADIR5,BFNQG,SEA_SURFACE_HEIGHT,0.935132,0.014810,69.443484,0.625617,118.899047,1.071163,36.701605
0,SWOT1NADIR5,MIOST,SEA_SURFACE_HEIGHT,0.939352,0.012452,89.130011,0.802973,131.211495,1.182086,10.704999
0,SWOT1NADIR5,MIOST,SEA_SURFACE_HEIGHT,0.939352,0.012452,89.130011,0.802973,131.211495,1.182086,10.704999
0,SWOT1NADIR5,DYMOST,SEA_SURFACE_HEIGHT,0.927870,0.020139,104.618228,0.942507,134.857913,1.214936,8.185237
0,SWOT1NADIR5,DYMOST,SEA_SURFACE_HEIGHT,0.927870,0.020139,104.618228,0.942507,134.857913,1.214936,8.185237
0,SWOT1NADIR5,DUACS,SEA_SURFACE_HEIGHT,0.920980,0.017840,108.557454,0.977995,137.573887,1.239404,12.078705


In [22]:
results.loc[cond1 & cond3].sort_values("λt [days]")

,Experiment,Method,Variable,µ(RMSE),σ(RMSE),iso λx [km],iso λx [degree],λx [km],λx [degree],λt [days]
0,SWOT1NADIR5,4DVARNET,SEA_SURFACE_HEIGHT,0.950371,0.009098,54.457253,0.490606,75.947262,0.684210,5.818690
0,SWOT1NADIR5,4DVARNET,SEA_SURFACE_HEIGHT,0.950371,0.009098,54.457253,0.490606,75.947262,0.684210,5.818690
0,SWOT1NADIR5,DYMOST,SEA_SURFACE_HEIGHT,0.927870,0.020139,104.618228,0.942507,134.857913,1.214936,8.185237
0,SWOT1NADIR5,DYMOST,SEA_SURFACE_HEIGHT,0.927870,0.020139,104.618228,0.942507,134.857913,1.214936,8.185237
0,SWOT1NADIR5,MIOST,SEA_SURFACE_HEIGHT,0.939352,0.012452,89.130011,0.802973,131.211495,1.182086,10.704999
0,SWOT1NADIR5,MIOST,SEA_SURFACE_HEIGHT,0.939352,0.012452,89.130011,0.802973,131.211495,1.182086,10.704999
0,SWOT1NADIR5,DUACS,SEA_SURFACE_HEIGHT,0.920980,0.017840,108.557454,0.977995,137.573887,1.239404,12.078705
0,SWOT1NADIR5,BFNQG,SEA_SURFACE_HEIGHT,0.935132,0.014810,69.443484,0.625617,118.899047,1.071163,36.701605
0,SWOT1NADIR5,BFNQG,SEA_SURFACE_HEIGHT,0.935132,0.014810,69.443484,0.625617,118.899047,1.071163,36.701605
0,SWOT1NADIR5,BFNQG,SEA_SURFACE_HEIGHT,0.935132,0.014810,69.443484,0.625617,118.899047,1.071163,36.701605
